In [ ]:
# imorting all the necessary libraries

import pandas as pd
import numpy as np
import psycopg2
import folium
import random as r
import spacy
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
from math import sin, cos, sqrt, atan2

In [ ]:
# connecting to the database

connection = psycopg2.connect(
    host = 'wh-pg-db.c8t3ik4p6f7t.us-east-2.rds.amazonaws.com',
    user = 'postgres',
    password = 'jNbZoN04gyzzyWdGT2ZU',
    database='whod'
    )
cursor=connection.cursor()

In [ ]:
# getting all the tables related to the candidate 

candidateSkills = pd.read_sql('select * from "CandidateSkills"', connection)
Skills = pd.read_sql('select * from "Skills"', connection)
SkillCategories = pd.read_sql('select * from "SkillCategories"', connection)


# convertingt the geolocation in to proper redable form and importing it

candidateLocation = pd.read_sql('select st_astext(location) from "Candidates"', connection)
candidates['location']=candidateLocation

In [ ]:
# extracting the longitude and latitude from the location column and placing them into seperate columns

long = candidates['location'].str.split('(').str[1].str.split(')').str[0].str.split(' ').str[0]
lat = candidates['location'].str.split('(').str[1].str.split(')').str[0].str.split(' ').str[1]


# converting the longitude and latitude to float 

candidates.long = candidates.long.astype('float')
candidates.lat = candidates.lat.astype('float')

In [ ]:
# extarcting meaningfull skills from the skills table and removing any garbage values

skills_list = Skills[Skills.name.str[4].notnull()].name
skills_list.index = range(0,len(skills_list))

In [ ]:
# filling missing skills in candidate table using random skills from table skills

for i in candidates[candidates.details.isnull()].index:
    candidates.details[i] = skills_list[r.randint(0,208)]


# filling latitude and longitude missing values using random values which are within the US coordinates 
    
for i in candidates[candidates.long.isnull()].index:
    candidates.long[i] = r.randint(34000000,47000000)/1000000
    candidates.lat[i] = r.randint(92000000,120000000)/1000000    

In [ ]:
# importing all the files for jobs

job = pd.read_sql('SELECT * FROM "Jobs"',connection)
site = pd.read_sql('SELECT * FROM "Sites"',connection)


# removing unnecessary columns from site

site = site.iloc[:,[0,2,3]]


# importing latitude and longitude in readable format

site_location = pd.read_sql('select st_astext(location) from "Sites"',connection)
site['location'] = site_location


# merging job with the respective location using siteId

job_merged = job.merge(site ,how = 'left',on = 'siteId')

In [ ]:
# extarcting the longitude and latitude from location

long = job_merged['location'].str.split('(').str[1].str.split(')').str[0].str.split(' ').str[0]
lat = job_merged['location'].str.split('(').str[1].str.split(')').str[0].str.split(' ').str[1]


# converting longitude and latitude to float 

job['long'] = long.astype('float')
job['lat'] = lat.astype('float')

In [ ]:
# filling in missing value of latitude and longitude with random values with the US coodinates

for i in job[job.long.isnull()].index:
    job.long[i] = r.randint(34000000,47000000)/1000000
    job.lat[i] = r.randint(92000000,120000000)/1000000

In [ ]:
# removing dublicate values and reorder the indexes based on Id

candidates = candidates.drop_duplicates(subset = 'candidateId')
candidates = candidates.sort_values('candidateId')
candidates.index = candidates.candidateId
job = job.sort_values('jobId')
job.index = job.jobId

In [ ]:
# Algorithm for clustering similar skill together based on similarity score

similarity_matrix = pd.DataFrame(index = candidates.candidateId, columns = candidates.candidateId)
for i in candidates.candidateId:
    print(i)
    for j in candidates.candidateId:
        similarity_matrix.loc[i,j] = nlp(candidates.details[j]).similarity(nlp(candidates.details[i]))
        
similarity_group = similarity_matrix[similarity_matrix>0.8].notnull().sum() # setting minimum similarity score at 0.8 

sm = similarity_matrix.copy()
i=0
while(i<x):
    indexes = sm[sm.iloc[:,i]>0.8][1:].index
    sm.drop(indexes,axis=1,inplace=True,errors = 'ignore')
    i=i+1
    x = sm.shape[1]
    
sm_group = sm[sm>0.8].notnull().sum()

In [ ]:
# Second variation of skill clustering algorithm, around 5-10 times faster

similarity_matrix = pd.DataFrame(index = candidates.candidateId, columns = candidates.candidateId)
i=0
k=0
while (k< similarity_matrix.shape[1]):
    i = similarity_matrix.columns[k]
    print(i)
    for j in candidates.candidateId:
        score = nlp(candidates.details[j]).similarity(nlp(candidates.details[i]))
        if((score>0.8) & (i!=j)): # condition to cluster only those which have similarity score greater than 0.8
            print('j',j)
            similarity_matrix.loc[j,i] = score
            similarity_matrix.drop(j,inplace=True,axis=1,errors='ignore')
    k=k+1    

In [ ]:
# function to calculate the distance between 2 coordinates 

from math import sin, cos, sqrt, atan2, radians
def distance(lat1,lon1,lat2,lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = ((sin(dlat/2))**2) + (cos(lat1) * cos(lat2) * ((sin(dlon/2))**2))
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance 

In [ ]:
# pre conversion of degrees to radians to save up time

candidates['lat_rad'] = candidates.lat.map(radians)
candidates['long_rad'] = candidates.long.map(radians)
job['lat_rad'] = job.lat.map(radians)
job['long_rad'] = job.long.map(radians)

In [ ]:
# concatenating candidates with candiates 130 times in a for loop to build 100,000 records for testing purpose

cand_all = candidates
for i in range(0,130):
    cand_all = pd.concat([cand_all,candidates],axis=0)
    

# concatenating job with job to get large number of jobs for testing purpose
    
jo = job
for i in range(0,1):
    jo = pd.concat([jo,job],axis=0    

In [ ]:
# reindexing the data and creating an empty dataframe to store all the scores

cand=cand_all
cand.index = range(0,len(cand))
jo.index = range(0,len(jo))
cand.candidateId = range(0,len(cand))
jo.jobId = range(0,len(jo))
df = pd.DataFrame(columns = sorted(jo.jobId), index = sorted(cand.candidateId))
df.replace(np.nan,0)

In [ ]:
# algorithm to match jobs with canddiates (version 1)

start_time = time.time()

df = pd.DataFrame(index = sorted(job.jobId), columns = sorted(candidates.candidateId))
w_skill = 0.3  # weightage for skill
w_distance = 0.7   # weightage for distance 
candidates.index = candidates.candidateId
job.index = job.jobId
for i in sorted(job.jobId)[:100]: # choosing 100 jobs for testing
    for j in sorted(candidates.candidateId)[:500]: # choosing 500 jobs for testing
        skill_score = nlp(candidates.loc[j,'details']).similarity(nlp(job.loc[i,'title'])) # calculating simialrity
        dist = distance(candidates.loc[j,'long'],candidates.loc[j,'lat'],job.loc[i,'long'],job.loc[i,'lat']) # calling distance function
        if(dist>500):  # condition for greater than 500 km
            distance_score = 0
        else:
            distance_score = 1/(pow(dist,dist*0.005))
        total_score = (skill_score*w_skill) + (distance_score*w_distance) # calculating total score
        df.loc[i,j] = total_score
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# algorithm to match jobs with canddiates (version 2)

start_time = time.time()

w_skill = 0.3
w_distance = 0.7

R = 6373.0
cand = cand_all.copy()
cand = cand[:10000] # selecting 10,000 candidates
for i in sorted(jo.jobId)[:1]:
    cand_filtered = cand[(cand.long<(jo.long[i]+1)) & (cand.long>(jo.long[i]-1)) &  # filtering based on coordinates
                         (cand.lat<(jo.lat[i]+1)) & (cand.lat>(jo.lat[i]-1))]
    for j in sorted(cand_filtered.candidateId):
        if((abs(cand.loc[j,'long']-jo.loc[i,'long']) + abs(cand.loc[j,'lat']-jo.loc[i,'lat'])) > 1): # calculating rough distance
            df.loc[j,i]  = 0
        else:
            skill_score = nlp(cand.loc[j,'details']).similarity(nlp(jo.loc[i,'title']))
            dist = distance(cand.loc[j,'long'],cand.loc[j,'lat'],jo.loc[i,'long'],jo.loc[i,'lat']) # calling distance function
            distance_score = 1/(pow(dist,dist*0.005))
            df.loc[j,i]  = (skill_score*w_skill) + (distance_score*w_distance)
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# algorithm to match jobs with canddiates (version 3)

start_time = time.time()

w_skill = 0.3
w_distance = 0.7

cand = cand_all.copy()
cand = cand[:50000] # selecting 50,000 jobs
i = sorted(jo.jobId)[2]
cand_filtered = cand[(cand.long<(jo.long[i]+0.5)) & (cand.long>(jo.long[i]-0.5)) & 
                     (cand.lat<(jo.lat[i]+0.5)) & (cand.lat>(jo.lat[i]-0.5))]  # filtering based on coordinates 

for j in sorted(cand_filtered.candidateId):
    skill_score = nlp(cand.details[j]).similarity(nlp(jo.title[i])) # calculating similarity score
    dist = distance(cand.long_rad[j],cand.lat_rad[j],jo.long_rad[i],jo.lat_rad[i]) # calling distance function 
    distance_score = 1/(pow(dist,dist*0.005))
    df.loc[j,i]  = (skill_score*w_skill) + (distance_score*w_distance) #  calculating total score
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# algorithm to match jobs with canddiates (version 4)

start_time = time.time()

w_skill = 0.3
w_distance = 0.7

close_value=[]
cand = cand_all.copy()
cand = cand[10000:100000] # choosing records from 10,000 to 100,000 for testing purpose
i = sorted(jo.jobId)[2]
cand_filtered = cand[(cand.long<(jo.long[i]+0.5)) & (cand.long>(jo.long[i]-0.5)) & 
                     (cand.lat<(jo.lat[i]+0.5)) & (cand.lat>(jo.lat[i]-0.5))]


# finding unique set of skills from all the candidates

values = cand_filtered['details'].value_counts()
values = values[values>3]
values.index


# calculating the similarity score for those unique set of skills

for k in range(0,len(values.index)):
    close_value.append(nlp(values.index[k]).similarity(nlp(job.title[1])))
close_skill = values.index[close_value.index(max(close_value))]
        
cand_filtered = cand_filtered[cand_filtered['details'] == close_skill]  # filtering a candidate based on identified skills 
    
for j in sorted(cand_filtered.candidateId):
    skill_score = max(close_value) # taking the already calculated similarity score
    dist = distance(cand.long[j],cand.lat[j],jo.long[i],jo.lat[i])
    distance_score = 1/(pow(dist,dist*0.005))
    df.loc[j,i]  = (skill_score*w_skill) + (distance_score*w_distance)
    
Id = df.loc[:,i].to_list().index(df.loc[:,i].max()) # extracting the Id of the best candidate
print(Id)
        
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# algorithm to match jobs with canddiates (version 4.2)

start_time = time.time()

w_skill = 0.3
w_distance = 0.7

close_skill=[]
cand = cand_all.copy()
cand = cand[100:100000]
i = sorted(jo.jobId)[140]
cand_filtered = cand[(cand.long<(jo.long[i]+0.5)) & (cand.long>(jo.long[i]-0.5)) & 
                     (cand.lat<(jo.lat[i]+0.5)) & (cand.lat>(jo.lat[i]-0.5))]

values = cand_filtered['details'].value_counts()
values = values[values>3]
values.index

for k in range(0,len(values.index)):
    score = nlp(values.index[k]).similarity(nlp(jo.title[i])) # finding similarity score for unique skills
    if(score>0.7):
        close_value.append(score)
        close_skill.append(values.index[k])
        
cand_filtered = cand_filtered[cand_filtered['details'].isin(close_skill)] # filtering candidates for a set of closely related skills
    
for j in sorted(cand_filtered.candidateId):
    skill_score = close_value[close_skill.index(cand_filtered.details[j])] # getting the pre calculated scores for skills
    dist = distance(cand.long[j],cand.lat[j],jo.long[i],jo.lat[i])
    distance_score = 1/(pow(dist,dist*0.005))
    df.loc[j,i]  = (skill_score*w_skill) + (distance_score*w_distance)
    
Id = df.loc[:,i].astype('float').nlargest(1000).index # finding top 1000 candidates for the given job
display(cand_all.loc[Id,:]) # displaying top 1000 candidates
        
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# for greater than one job buiding a dataframe to store matched candidates with the respective jobs

match = pd.DataFrame(index = range(0,len(job.jobId)),columns=(['jobId','candidateId']))
match.jobId = sorted(job.jobId)
df_array = np.array(df)
match.index = match.jobId

In [ ]:
# storing matched values in table match

df1 = df.copy()
for i in sorted(match.jobId):
        Id = df1.columns[df1.loc[i,:].tolist().index(max(df1.loc[i,:]))]
        match.loc[i,'candidateId'] = Id
        df1.drop(Id,axis = 1,inplace=True)

In [ ]:
# adding candidate latitude longitude skill columns to table match for testing purpose
 
candidates.index = candidates.candidateId
match.index = match['candidateId']
match['candidate_long'] = candidates.loc[match['candidateId'],'long']
match['candidate_lat'] = candidates.loc[match['candidateId'],'lat']
match['candidate_skill'] = candidates.loc[match['candidateId'],'details']

In [ ]:
# adding job latitude longitude skill columns to table match for testing purpose

job.index = job.jobId
match.index = match['jobId']
match['job_long'] = job.loc[match['jobId'],'long']
match['job_lat'] = job.loc[match['jobId'],'lat']
match['job_skill'] = job.loc[match['jobId'],'title']

In [ ]:
#  adding distance and skill score to table match

skill_score = []
for i in range(0,len(match)):
    skill_score.append(nlp(job.title[match['jobId'][i]]).similarity(nlp(candidates.details[match['candidateId'][i]])))
    
dist=[]
for i in range(0,len(match)):
    dist.append(distance(match.candidate_long[i],match.candidate_lat[i],match.job_long[i],match.job_lat[i]))

In [ ]:
# adding weighted skill score weighted distance score total score to table match

match['skill_score'] = skill_score
match['weighted_skill_score'] = np.array(skill_score)*0.3
match['distance'] = dist

dist = np.array(dist)
match['distance_score'] = np.round(1/(pow(dist,dist*0.005)),10)

match['weighted_distance_score'] = (np.round(1/(pow(dist,dist*0.005)),10))*0.7

match['total_score'] = (np.array(skill_score)*0.3) + ((np.round(1/(pow(dist,dist*0.005)),10))*0.7)